from utils import write_transform_to_file
write_transform_to_file('adult') # needs to be run only once per dataset

#### Load the data transformer and the transformed data

In [1]:
import json
import itertools
import os
import torch

from ctgan.synthesizers.ae_gan import CTGANV2
from benchmark import train_model
from utils import load_transform_from_file, read_target_idx, sample_fake_adult


AE_TYPES = ['vanilla', 'denoising', 'vae', 'ee']
HP = {
    'ae_dim' : [(256,128), (256,128,64), (256,128,64,32)], # (1)
    'embedding_dim' : [64, 128, 256], #(2)
    'gan_layers': [(256,256), (256,256,256,256)], # (3)
    'discriminator_steps': [1, 5], # (4)
    'pac': [4, 8, 16], # (4)
    'gan_lr': [1e-5, 2e-4], # (5)
    'gan_batch_epoch': [(256,150), (512,300)], # (5)
    'ae_lr': [1e-4, 2e-4, 1e-3], # (5)
    'ae_batch_epoch': [(256,50), (512,100)], # (5)
}
if os.path.exists("adult_opt_hp.json"):
    with open("adult_opt_hp.json", 'r') as f:
        hp_vals = json.load(f)
else:
    hp_vals = {k:dict.fromkeys(HP.keys()) for k in AE_TYPES}
    for method in hp_vals.keys():
        hp_vals[method]['dstep_pac'] = hp_vals[method].pop('discriminator_steps')
        del hp_vals[method]['pac']
        hp_vals[method]['gbe_aeb'] = hp_vals[method].pop('gan_lr')
        del hp_vals[method]['gan_batch_epoch']
        del hp_vals[method]['ae_lr']
        del hp_vals[method]['ae_batch_epoch']

# Load the transformed data
dt, td = load_transform_from_file('adult')
target_idx = read_target_idx('adult')

Data transformer loaded
Transformed data loaded


#### (1) Find the optimal AE dimension

In [2]:
hp = 'ae_dim'

for ae_type in AE_TYPES:
    hp_scores = dict()
    for v in HP[hp]:
        k = '_'.join(str(i) for i in v)
        if not os.path.exists(f"../../models/ae_gan_{ae_type}_{hp}_{k}.pth"):
            # Train the AE_GAN
            print(f"Training CTGAN: {ae_type} - {hp} - {v}")
            ae_gan = CTGANV2(ae_type=ae_type, ae_dim=v)
            ae_gan.fit(td['train'],dt=dt,is_transformed=True, target_index=target_idx)
            print("Training Complete!")
            # save model
            ae_gan.save(f"../../models/ae_gan_{ae_type}_{hp}_{k}.pth")
        else:
            print(f"Loading CTGAN: {ae_type} - {hp} - {v}")
            ae_gan = CTGANV2().load(f"../../models/ae_gan_{ae_type}_{hp}_{k}.pth")

        best_score = torch.tensor([0.0])
        try:
            # Sample fake data
            fake_data, real_data = sample_fake_adult(dt,td,ae_gan)
            
            print("Benchmarking on MLP100")
            # Train the MLP on fake validation data
            for _ in range(3):
                test_score = train_model(
                                fake_data[0],
                                fake_data[1],
                                fake_data[2],
                                fake_data[3],
                                real_data[0],
                                real_data[1],
                                batch_size=256,
                                num_epochs=100,
                                model_type="classification",
                                verbose=False,
                                scorer_type="accuracy",
                            )
                if test_score > best_score:
                    best_score = test_score
            print("Benchmark Complete!")
        except:
            pass
        hp_scores[k] = best_score.item()
    
    hp_vals[ae_type][hp] = hp_scores
    with open("adult_opt_hp.json", 'w') as f:
        f.write(json.dumps(hp_vals))

Loading CTGAN: vanilla - ae_dim - (256, 128)
Benchmarking on MLP100


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.8885135054588318


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.9006449580192566


...

Best validation score: 1.0


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 1.0


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 1.0
Benchmark Complete!
<class 'float'>
{
    "vanilla": {
        "ae_dim": {
            "256_128": 0.7751842737197876,
            "256_128_64": 0.7592137455940247,
            "256_128_64_32": 0.7598280310630798
        },
        "embedding_dim": null,
        "gan_layers": null,
        "discriminator_steps": null,
        "pac": null,
        "gan_lr": null,
        "gan_batch_epoch": null,
        "ae_lr": null,
        "ae_batch_epoch": null
    },
    "denoising": {
        "ae_dim": {
            "256_128": 0.7714987993240356,
            "256_128_64": 0.30850738286972046,
            "256_128_64_32": 0.7624385952949524
        },
        "embedding_dim": null,
        "gan_layers": null,
        "discriminator_steps": null,
        "pac": null,
        "gan_lr": null,
        "gan_batch_epoch": null,
        "ae_lr": null,
        "ae_batch_epoch": null
    },
    "vae": {
        "ae_dim": {
            "256_128": 0.7592137455940247,
            "256

#### (2) Find the optimal G latent dimension

In [8]:
hp = 'embedding_dim'

for ae_type in AE_TYPES:
    hp_scores = dict()
    printable_ae_dim = max(hp_vals[ae_type]['ae_dim'].items(), key=lambda x: x[1])[0]
    ae_dim = tuple(map(int, printable_ae_dim.split('_')))
    for v in HP[hp]:
        if not os.path.exists(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_{hp}_{v}.pth"):
            # Train the AE_GAN
            print(f"Training CTGAN: {ae_type} - ae_dim - {printable_ae_dim} - {hp} - {v}")
            ae_gan = CTGANV2(ae_type=ae_type, ae_dim=ae_dim, embedding_dim=v)
            ae_gan.fit(td['train'],dt=dt,is_transformed=True, target_index=target_idx)
            print("Training Complete!")
            # save model
            ae_gan.save(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_{hp}_{v}.pth")
        else:
            print(f"Loading CTGAN: {ae_type} - ae_dim - {printable_ae_dim} - {hp} - {v}")
            ae_gan = CTGANV2().load(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_{hp}_{v}.pth")

        best_score = torch.tensor([0.0])
        try:
            # Sample fake data
            fake_data, real_data = sample_fake_adult(dt,td,ae_gan)
            
            print("Benchmarking on MLP100")
            # Train the MLP on fake validation data
            for _ in range(3):
                test_score = train_model(
                                fake_data[0],
                                fake_data[1],
                                fake_data[2],
                                fake_data[3],
                                real_data[0],
                                real_data[1],
                                batch_size=256,
                                num_epochs=100,
                                model_type="classification",
                                verbose=False,
                                scorer_type="accuracy",
                            )
                if test_score > best_score:
                    best_score = test_score
            print("Benchmark Complete!")
        except:
            pass
        hp_scores[v] = best_score.item()
    
    hp_vals[ae_type][hp] = hp_scores
    with open("adult_opt_hp.json", 'w') as f:
        f.write(json.dumps(hp_vals))

Loading CTGAN: vanilla - ae_dim - 256_128 - embedding_dim - 64
Benchmarking on MLP100


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.937039315700531


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.9348894357681274


...

Best validation score: 1.0


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 1.0


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 1.0
Benchmark Complete!
Loading CTGAN: ee - ae_dim - 256_128_64 - embedding_dim - 256


#### (3) Find the optimal no G/D layers

In [10]:
hp = 'gan_layers'

for ae_type in AE_TYPES:
    hp_scores = dict()
    printable_ae_dim = max(hp_vals[ae_type]['ae_dim'].items(), key=lambda x: x[1])[0]
    ae_dim = tuple(map(int, printable_ae_dim.split('_')))
    embedding_dim = int(max(hp_vals[ae_type]['embedding_dim'].items(), key=lambda x: x[1])[0])
    for v in HP[hp]:
        k = '_'.join(str(i) for i in v)
        if not os.path.exists(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_emb_{embedding_dim}_{hp}_{v}.pth"):
            # Train the AE_GAN
            print(f"Training CTGAN: {ae_type} - ae_dim - {printable_ae_dim} - embed_dim - {embedding_dim} - {hp} - {v}")
            ae_gan = CTGANV2(ae_type=ae_type, ae_dim=ae_dim,
                             embedding_dim=embedding_dim, generator_dim=v, discriminator_dim=v)
            ae_gan.fit(td['train'], dt=dt, is_transformed=True, target_index=target_idx)
            print("Training Complete!")
            # save model
            ae_gan.save(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_emb_{embedding_dim}_{hp}_{v}.pth")
        else:
            print(f"Loading CTGAN: {ae_type} - ae_dim - {printable_ae_dim} - embed_dim - {embedding_dim} - {hp} - {v}")
            ae_gan = CTGANV2().load(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_emb_{embedding_dim}_{hp}_{v}.pth")

        best_score = torch.tensor([0.0])
        try:
            # Sample fake data
            fake_data, real_data = sample_fake_adult(dt,td,ae_gan)
            
            print("Benchmarking on MLP100")
            # Train the MLP on fake validation data
            for _ in range(3):
                test_score = train_model(
                                fake_data[0],
                                fake_data[1],
                                fake_data[2],
                                fake_data[3],
                                real_data[0],
                                real_data[1],
                                batch_size=256,
                                num_epochs=100,
                                model_type="classification",
                                verbose=False,
                                scorer_type="accuracy",
                            )
                if test_score > best_score:
                    best_score = test_score
            print("Benchmark Complete!")
        except:
            pass
        hp_scores[k] = best_score.item()
    
    hp_vals[ae_type][hp] = hp_scores
    with open("adult_opt_hp.json", 'w') as f:
        f.write(json.dumps(hp_vals))

Training CTGAN: vanilla - ae_dim - 256_128 - embed_dim - 64 - gan_layers - (256, 256)
Training Complete!
Benchmarking on MLP100


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.9887899160385132


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.9874078631401062


...

Best validation score: 0.9989250898361206


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.9987714886665344


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.9989250898361206
Benchmark Complete!


#### (4) Find the optimal no D_steps and pac

In [2]:
hp = 'dstep_pac'
dstep_pac = list(itertools.product(HP['discriminator_steps'], HP['pac']))

for ae_type in AE_TYPES:
    hp_scores = dict()
    printable_ae_dim = max(hp_vals[ae_type]['ae_dim'].items(), key=lambda x: x[1])[0]
    ae_dim = tuple(map(int, printable_ae_dim.split('_')))
    embedding_dim = int(max(hp_vals[ae_type]['embedding_dim'].items(), key=lambda x: x[1])[0])
    printable_gan_layers = max(hp_vals[ae_type]['gan_layers'].items(), key=lambda x: x[1])[0]
    gan_layers = tuple(map(int, printable_gan_layers.split('_')))
    for v1, v2 in dstep_pac:
        k = '_'.join(map(str,[v1,v2]))
        if not os.path.exists(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_emb_{embedding_dim}_gla_{gan_layers}_dstep_{v1}_pac_{v2}.pth"):
            # Train the AE_GAN
            print(f"Training CTGAN: {ae_type} - ae_dim - {printable_ae_dim} - embed_dim - {embedding_dim} - gan_layers - {printable_gan_layers} - dstep - {v1} - pac - {v2}")
            ae_gan = CTGANV2(ae_type=ae_type, ae_dim=ae_dim, embedding_dim=embedding_dim,
                                generator_dim=gan_layers, discriminator_dim=gan_layers,
                                discriminator_steps=v1, pac=v2)
            ae_gan.fit(td['train'], dt=dt, is_transformed=True, target_index=target_idx)
            print("Training Complete!")
            # save model
            ae_gan.save(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_emb_{embedding_dim}_gla_{gan_layers}_dstep_{v1}_pac_{v2}.pth")
        else:
            print(f"Loading CTGAN: {ae_type} - ae_dim - {printable_ae_dim} - embed_dim - {embedding_dim} - gan_layers - {printable_gan_layers} - dstep - {v1} - pac - {v2}")
            ae_gan = CTGANV2().load(f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_emb_{embedding_dim}_gla_{gan_layers}_dstep_{v1}_pac_{v2}.pth")

        best_score = torch.tensor([0.0])
        try:
            # Sample fake data
            fake_data, real_data = sample_fake_adult(dt,td,ae_gan)
            
            print("Benchmarking on MLP100")
            # Train the MLP on fake validation data
            for _ in range(3):
                test_score = train_model(
                                fake_data[0],
                                fake_data[1],
                                fake_data[2],
                                fake_data[3],
                                real_data[0],
                                real_data[1],
                                batch_size=256,
                                num_epochs=100,
                                model_type="classification",
                                verbose=False,
                                scorer_type="accuracy",
                            )
                if test_score > best_score:
                    best_score = test_score
            print("Benchmark Complete!")
        except:
            pass
        hp_scores[k] = best_score.item()
    
    hp_vals[ae_type][hp] = hp_scores
    with open("adult_opt_hp.json", 'w') as f:
        f.write(json.dumps(hp_vals))

Training CTGAN: vanilla - ae_dim - 256_128 - embed_dim - 64 - gan_layers - 256_256 - dstep - 1 - pac - 4
Training Complete!
Benchmarking on MLP100


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.9390356540679932


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 0.9269042015075684


...

Best validation score: 1.0


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 1.0


  0%|          | 0/100 [00:00<?, ?it/s]

Best validation score: 1.0
Benchmark Complete!


#### (5) Find the optimal GAN LR, GAN BatchSize-Epoch, AE LR, AE BatchSize-Epoch

In [9]:
from tqdm.notebook import tqdm
hp = 'gbe_aeb'
gbe_aeb = list(itertools.product(HP['gan_lr'], HP['gan_batch_epoch'], HP['ae_lr'], HP['ae_batch_epoch']))

for ae_type in tqdm(AE_TYPES):
    hp_scores = dict()
    printable_ae_dim = max(hp_vals[ae_type]['ae_dim'].items(), key=lambda x: x[1])[0]
    ae_dim = tuple(map(int, printable_ae_dim.split('_')))
    embedding_dim = int(max(hp_vals[ae_type]['embedding_dim'].items(), key=lambda x: x[1])[0])
    printable_gan_layers = max(hp_vals[ae_type]['gan_layers'].items(), key=lambda x: x[1])[0]
    gan_layers = tuple(map(int, printable_gan_layers.split('_')))
    d_step, pac = list(map(int,max(hp_vals[ae_type]['dstep_pac'].items(), key=lambda x: x[1])[0].split('_')))
    for v1, v2, v3, v4 in tqdm(gbe_aeb):
        pv2 = '_'.join(map(str,v2))
        pv4 = '_'.join(map(str,v4))
        k = '_'.join(map(str,[v1,pv2,v3,pv4]))
        mpath = f"../../models/ae_gan_{ae_type}_ae_dim_{printable_ae_dim}_emb_{embedding_dim}_gla_{gan_layers}_dstep_{d_step}_pac_{pac}_glr_{v1}_gbe_{pv2}_alr_{v3}_aeb_{pv4}.pth"
        if not os.path.exists(mpath):
            # Train the AE_GAN
            print(f"Training CTGAN: {ae_type} - ae_dim - {printable_ae_dim} - embed_dim - {embedding_dim} - gan_layers - {printable_gan_layers} - dstep - {d_step} - pac - {pac} - gan_lr - {v1} - gan_batch_epoch - {pv2} - ae_lr - {v3} - ae_batch_epoch - {pv4}")
            ae_gan = CTGANV2(ae_type=ae_type, ae_dim=ae_dim, embedding_dim=embedding_dim,
                                generator_dim=gan_layers, discriminator_dim=gan_layers,
                                discriminator_steps=d_step, pac=pac, generator_lr=v1, discriminator_lr=v1,
                                batch_size=v2[0], epochs=v2[1], autoencoder_lr=v3, ae_batch_size=v4[0], ae_epochs=v4[1])
            ae_gan.fit(td['train'], dt=dt, is_transformed=True, target_index=target_idx)
            print("Training Complete!")
            # save model
            ae_gan.save(mpath)
        else:
            print(f"Loading CTGAN: {ae_type} - ae_dim - {printable_ae_dim} - embed_dim - {embedding_dim} - gan_layers - {printable_gan_layers} - dstep - {d_step} - pac - {pac} - gan_lr - {v1} - gan_batch_epoch - {pv2} - ae_lr - {v3} - ae_batch_epoch - {pv4}")
            ae_gan = CTGANV2().load(mpath)

        best_score = torch.tensor([0.0])
        try:
            # Sample fake data
            fake_data, real_data = sample_fake_adult(dt,td,ae_gan)
            
            print("Benchmarking on MLP100")
            # Train the MLP on fake validation data
            for _ in range(3):
                test_score = train_model(
                                fake_data[0],
                                fake_data[1],
                                fake_data[2],
                                fake_data[3],
                                real_data[0],
                                real_data[1],
                                batch_size=256,
                                num_epochs=100,
                                model_type="classification",
                                verbose=False,
                                scorer_type="accuracy",
                            )
                if test_score > best_score:
                    best_score = test_score
            print("Benchmark Complete!")
        except:
            pass
        hp_scores[k] = best_score.item()
    
    hp_vals[ae_type][hp] = hp_scores
    with open("adult_opt_hp.json", 'w') as f:
        f.write(json.dumps(hp_vals))

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/24 [00:00<?, ?it/s]

Training CTGAN: vanilla - ae_dim - 256_128 - embed_dim - 64 -                   gan_layers - 256_256 - dstep - 1e-05 - pac - (256, 150) -                   gan_lr - 1e-05 - gan_batch_epoch - 256_150 - ae_lr - 0.0001 - ae_batch_epoch - 256_50


KeyboardInterrupt: 